In [1]:
import langchain
from langchain.llms import OpenAI
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter , RecursiveJsonSplitter
from dotenv import load_dotenv
load_dotenv()
import os 
from langchain.graphs import Neo4jGraph
import langchain_experimental
from langchain_experimental.graph_transformers import LLMGraphTransformer
import os
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from typing import Tuple, List, Optional
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


Clean the dataset to filter out only the noise disturbance classes of conversation.

In [8]:
conversation = []
import json
import pandas as pd
dataset = pd.DataFrame()
with open(r"./user4.jsonl", 'r') as file:
    for line in file:
        data = json.loads(line)
        dataset = dataset._append(data , ignore_index = True)


In [9]:
print(set(dataset["type"]))

{'SuspiciousActivity', 'NoiseDisturbance', 'DrugsAlcohol', 'AccidentTrafficParking', 'TheftLostItem', 'HarassmentAbuse', 'MentalHealth', 'unknown', 'EmergencyMessage', 'FacilitiesMaintenance'}


In [10]:
noise_disturbance_db = (dataset[dataset["type"] == "NoiseDisturbance"])

In [11]:
conversation = ""
cnt = 0
for iter in (noise_disturbance_db["history"][20:30]):
    tc = ""
    for item in iter:
      conversation += (str(item[0])  + " : " +  str( item[1]) + ". ")


conversations = {"conversations"  : conversation}
conversations
print(len(conversations["conversations"]))

4931


In [12]:
conversations

{'conversations': "User : Hi, could someone please come ask the person in Moffitt ### to turn off their music it is disturbing the people downstairs, beside him, and across the hall. People have asked him to turn it down, however he would not, and turned it up.. Dispatcher : I will notify the police, thank you.. User : Thank you. User : Hi, could someone please come ask the person in Moffitt ### to turn off their music it is disturbing the people downstairs, beside him, and across the hall. People have asked him to turn it down, however he would not, and turned it up.. Dispatcher : I will notify the police, thank you.. User : Thank you. User : Hi, was someone able to be sent out?. Dispatcher : Yes, they are onscene at this time.. Dispatcher : Would you like to speak to the officer?. User : No, but thank you. User : Did they stop and talk to him because it is still extremely loud. Dispatcher : They are still onscene.. User : Okay, thank you. User : I'm not sure who to contact about this

In [13]:
#Skip this step
!pip install OpenAI

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4-turbo-preview",
  messages=[
    {"role": "system", "content": "You are an assistant that generates a knowledge graph from the provided dataset."},
    {"role": "user", "content": conversation}]
)

print(completion.choices[0].message)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
ChatCompletionMessage(content='Given the dataset, here are the key entities and their relations:\n\n1. **Locations:**\n   - Moffitt\n   - University of California, Berkeley\n   - Shrine and 10th\n   - Los Angeles\n   - 123 Main St\n   - ### Tremont\n   - New York City\n\n2. **Incidents:**\n   - Noise Complaints\n     - Loud music disturbing residents in Moffitt\n     - Intoxicated individuals potentially damaging cars at Shrine and 10th\n     - Noisy and possibly drunk people in the vicinity of ACME Corporation and Los Angeles\n     - Loud music and yelling near 123 Main St, New York City\n   - Email Reply-All Issue at the University of California, Berkeley\n   - Publ

In [15]:
#load the document of the conversation.
import json
f = open('./convo_intent.json')
convo_intent = json.load(f)
loader = TextLoader(conversation)
raw_documents = loader.load
raw_documents


<bound method BaseLoader.load of <langchain_community.document_loaders.text.TextLoader object at 0x7c29f224ba30>>

In [16]:
conversation


"User : Hi, could someone please come ask the person in Moffitt ### to turn off their music it is disturbing the people downstairs, beside him, and across the hall. People have asked him to turn it down, however he would not, and turned it up.. Dispatcher : I will notify the police, thank you.. User : Thank you. User : Hi, could someone please come ask the person in Moffitt ### to turn off their music it is disturbing the people downstairs, beside him, and across the hall. People have asked him to turn it down, however he would not, and turned it up.. Dispatcher : I will notify the police, thank you.. User : Thank you. User : Hi, was someone able to be sent out?. Dispatcher : Yes, they are onscene at this time.. Dispatcher : Would you like to speak to the officer?. User : No, but thank you. User : Did they stop and talk to him because it is still extremely loud. Dispatcher : They are still onscene.. User : Okay, thank you. User : I'm not sure who to contact about this, but please tell 

In [14]:
!pip install langchain-experimental
!pip install openai
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.0 MB/s eta 0:00:00


In [15]:
#Run a text splitter
!pip install tiktoken
text_splitter = TokenTextSplitter()
documents = text_splitter.create_documents(conversation)
documents

[Document(page_content='U'),
 Document(page_content='s'),
 Document(page_content='e'),
 Document(page_content='r'),
 Document(page_content=' '),
 Document(page_content=':'),
 Document(page_content=' '),
 Document(page_content='H'),
 Document(page_content='i'),
 Document(page_content=','),
 Document(page_content=' '),
 Document(page_content='c'),
 Document(page_content='o'),
 Document(page_content='u'),
 Document(page_content='l'),
 Document(page_content='d'),
 Document(page_content=' '),
 Document(page_content='s'),
 Document(page_content='o'),
 Document(page_content='m'),
 Document(page_content='e'),
 Document(page_content='o'),
 Document(page_content='n'),
 Document(page_content='e'),
 Document(page_content=' '),
 Document(page_content='p'),
 Document(page_content='l'),
 Document(page_content='e'),
 Document(page_content='a'),
 Document(page_content='s'),
 Document(page_content='e'),
 Document(page_content=' '),
 Document(page_content='c'),
 Document(page_content='o'),
 Document(page

Create a graph database in the vector database

In [16]:
!pip install neo4j

url = os.getenv("NEO4J_URI")
username =os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [22]:

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo-preview")

In [104]:
#convert the docucments to a graph structure. Skip its aldready has been done

documents = [Document(page_content=conversation)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [17]:
!pip install yfiles_jupyter_graphs
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t"


def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.getenv("NEO4J_URI"),
        auth = (os.getenv("NEO4J_USERNAME"),
                os.getenv("NEO4J_PASSWORD")))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s eta 0:00:00


GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
showGraph()

In [18]:
!pip install langchain_community
!pip install langchain_openai

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [23]:
# Retriever


graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [24]:
entity_chain.invoke({"question": "How to resolve noise disturbance"})

Entities(names=['noise disturbance'])

In [109]:
graph.query("SHOW VECTOR INDEX")

[{'id': 4,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Document'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 3, 31, 21, 12, 21, 236000000, tzinfo=<UTC>),
  'readCount': 6}]

In [25]:

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [26]:
print(structured_retriever("What is the name of the dispatcher"))


In [27]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [28]:


_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [122]:
#do not run this
class ChatPromptTemplate:
    def __init__(self, template, memory=None, messagesplaceholder=None):
        self.template = template
        self.memory = memory
        self.messagesplaceholder = messagesplaceholder

    @classmethod
    def from_template(cls, template, memory=None, messagesplaceholder=None):
        return cls(template, memory, messagesplaceholder)


In [29]:
!pip install langchain_core

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo-preview")
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [49]:
mem = []
prev_context = """
This is the previous history of the conversation and respond to the last line in the conversation.

"""

def prompt_from_memory(input , mem , prev_context):
  mem.append(("user" , str(input)))
  for conv in mem:
    prev_context += (str(conv[0]) + " : " + str(conv[1])) + " "

  x = chain.invoke({"question" : str(input)})

  mem.append(("bot" , str(x)))
  print(x)
  print(prev_context)

In [50]:
prompt_from_memory( "There is noise disturbance in my locality. Could you please help me with?" , mem , prev_context)

Search query: There is noise disturbance in my locality. Could you please help me with?
Yes, I can help with noise disturbances. Please provide details such as your location and the nature of the disturbance, and I will notify the appropriate authorities for you.

This is the previous history of the conversation and respond to the last line in the conversation.

user : There is noise disturbance in my locality. Could you please help me with? 
